Latex Macros:
$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\A}[1]{\mathbf{A}_{#1}}
\newcommand{\Ai}[1]{\mathbf{A}^{#1}}
$$

In [1]:
macro javascript_str(s) display("text/javascript", s); end
javascript"""
    MathJax.Hub.Config({
      TeX: { equationNumbers: { autoNumber: "AMS" } }
    });
    MathJax.Hub.Queue( 
        ["resetEquationNumbers",MathJax.InputJax.TeX], 
        ["PreProcess",MathJax.Hub], 
        ["Reprocess",MathJax.Hub] 
    );
"""

Mixed Linear Model
=======================================

Consider the mixed linear model

\begin{equation}
\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \mathbf{Zu} + \mathbf{e}, 
\label{eq.mixedModel}
\end{equation}

where $\boldsymbol{\beta}$ is a vector of fixed effects, $\mathbf{X}$ is an observed matrix that relates $\boldsymbol{\beta}$ to $\mathbf{y}$, $\mathbf{u}$ is a vector of multivariate normal random effects with null means and covariance matrix $\mathbf{G}\sigma^2_u$, and $\mathbf{e}$ is a vector of multivariate normal residuals with null means and covariance matrix $\mathbf{R}\sigma^2_e$. The fixed effects are usually assigned a flat prior distribution, and the matrices $\mathbf{G}$ and $\mathbf{R}$ are assumed to be known. The variance components $\sigma^2_u$ and $\sigma^2_e$ will be assigned scaled inverted chi-square prior distributions with scale parameter $S_{u}^{2}$ and degrees of freedom $\nu_{u}$ for $\sigma^2_u$ and parameters $S_{e}^{2}$ and $\nu_{e}$ for $\sigma^2_e$. 

Then, the joint
posterior for all the unknowns, denoted by $\boldsymbol{\theta}$, is

\begin{equation}
\begin{aligned}
f(\boldsymbol{\theta}|\mathbf{y}) 
& \propto f(\mathbf{y}|\boldsymbol{\theta})f(\boldsymbol{\theta})\\
& \propto \left(\sigma_{e}^{2}\right)^{-n/2}
\exp\left\{ 
-\frac{(\mathbf{y} - \mathbf{X}\boldsymbol{\beta} - \mathbf{Zu})'\mathbf{R}^{-1} 
       (\mathbf{y} - \mathbf{X}\boldsymbol{\beta} - \mathbf{Zu})}{2\sigma_{e}^{2}}
\right\} \\
& \times \left(\sigma_{u}^{2}\right)^{-k/2}
\exp \left\{ 
-\frac{\mathbf{u}'\mathbf{G}^{-1}\mathbf{u}}
      {2\sigma_{u}^{2}}
\right\} \\
& \times (\sigma_{u}^{2})^{-(2+\nu_{u})/2}\exp\left\{ -\frac{\nu_{u}S_{u}^{2}}{2\sigma_{u}^{2}}\right\} \\
& \times (\sigma_{e}^{2})^{-(2+\nu_{e})/2}\exp\left\{ -\frac{\nu_{e}S_{e}^{2}}{2\sigma_{e}^{2}}\right\} .\\
\\
\end{aligned}
\end{equation}

We show below that the full-conditional distribution for the location parameters in the model, $\boldsymbol{\theta}_1' = [\boldsymbol{\beta}', \mathbf{u}']$, is normal with mean given by the solution to:

\begin{equation}
\begin{bmatrix}
\mathbf{X}'\mathbf{R}^{-1}\mathbf{X} & \mathbf{X}'\mathbf{R}^{-1}\mathbf{Z} \\
\mathbf{Z}'\mathbf{R}^{-1}\mathbf{X} & \mathbf{Z}'\mathbf{R}^{-1}\mathbf{Z} + \frac{\sigma_{e}^{2}}{\sigma_u^2}\mathbf{G}^{-1}
\end{bmatrix}
\begin{bmatrix}
\hat{\bs{\beta}}\\
\hat{\mbf{u}}
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{X}'\mathbf{R}^{-1}\mathbf{y}\\
\mathbf{Z}'\mathbf{R}^{-1}\mathbf{y}
\end{bmatrix},
\label{eq.MME}
\end{equation}

and variance-covariance matrix given by the inverse of the coefficient matrix of (\ref{eq.MME}).

For convenience, we will write the above equation as:
$$
(\mathbf{W}'\mathbf{R}^{-1}\mathbf{W} + \frac{\sigma_{e}^{2}}{\sigma_u^2}\mathbf{D})\hat{\bs{\theta}}_1=\mathbf{W}'\mathbf{R}^{-1}\mathbf{y},
$$

where $\mathbf{W} = \begin{bmatrix}\mathbf{X} & \mathbf{Z}\end{bmatrix}$ and $\mathbf{D}$ is a block diagonal matrix that adds zero to the diagonals of $\mathbf{X}'\mathbf{R}^{-1}\mathbf{X}$ 
and $\mathbf{G}^{-1}$ to $\mathbf{Z}'\mathbf{R}^{-1}\mathbf{Z}$. 

The full-conditional posterior for $\bs{\theta}_1$ is 

$$\begin{aligned}
f(\boldsymbol{\theta}_1|\mathbf{y},\sigma_{u}^{2},\sigma_{e}^{2})
&  =\frac{f(\mathbf{y}|\boldsymbol{\theta}_1,\sigma_{u}^{2},\sigma_{e}^{2})f(\mathbf{u}|\sigma_{u}^{2})f(\sigma_{u}^{2})f(\sigma_{e}^{2})}{f(\mathbf{y},\sigma_{u}^{2},\sigma_{e}^{2})}\\ 
& \propto f(\mathbf{y}|\boldsymbol{\theta}_1,\sigma_{u}^{2},\sigma_{e}^{2})f(\mathbf{u}|\sigma_{u}^{2})\\
& \propto \left(\sigma_{e}^{2}\right)^{-n/2}
\exp\left\{ 
-\frac{(\mathbf{y}-\mathbf{\mathbf{W}}\boldsymbol{\theta}_1)'\mathbf{R}^{-1}(\mathbf{y}-\mathbf{W}\boldsymbol{\theta}_1)}
      {2\sigma_{e}^{2}}\right\} \\
& \times \left(\sigma_u^2\right)^{-k/2}
\exp\left\{ 
-\frac{\mathbf{u}'\mathbf{G}^{-1}\mathbf{u}}
      {2\sigma_{u}^{2}}
\right\} \\
& \propto \exp\left\{ -\frac{(\mathbf{y}-\mathbf{\mathbf{W}}\boldsymbol{\theta}_1)'\mathbf{R}^{-1}(\mathbf{y}-\mathbf{W}\boldsymbol{\theta}_1)+
\frac{\sigma_e^2}{\sigma_u^2}\mathbf{u}'\mathbf{G}^{-1}\mathbf{u}}{2\sigma_{e}^{2}}\right\} \\
& \propto \exp\left\{ -\frac{\mathbf{y}'\mathbf{R}^{-1}\mathbf{y}-2\mathbf{y}'\mathbf{R}^{-1}\mathbf{W}\boldsymbol{\theta}_1
+\bs{\theta}_1'(\mathbf{W}'\mathbf{R}^{-1}\mathbf{W}+\frac{\sigma_{e}^{2}}{\sigma_u^2}\mathbf{D})\boldsymbol{\theta}_1}{2\sigma_{e}^{2}}\right\} \\
& \propto  
\exp\left\{ -\frac{\mathbf{y}'\mathbf{R}^{-1}\mathbf{y} -
(\bs{\theta}_1 - \hat{\bs{\theta}}_1)'(\mathbf{W}'\mathbf{R}^{-1}\mathbf{W} 
+ \frac{\sigma_e^2}{\sigma_u^2}\mathbf{D})(\bs{\theta}_1 - \hat{\bs{\theta}}_1) - \hat{\bs{\theta}}_1'(\mathbf{W}'\mathbf{R}^{-1}\mathbf{W} + \frac{\sigma_e^2}{\sigma^2_u}\mathbf{D})\hat{\bs{\theta}}_1}{2\sigma^2_e}\right\} \\
& \propto  \exp\left\{ -\frac{(\bs{\theta}_1 - \hat{\bs{\theta}}_1)'(\mathbf{W}'\mathbf{R}^{-1}\mathbf{W} 
+ \frac{\sigma_e^2}{\sigma_u^2}\mathbf{D})(\bs{\theta}_1 - \hat{\bs{\theta}}_1)}{2\sigma_{e}^{2}}\right\} ,
\end{aligned}$$

which can be recognized as a nromal distribution with mean $\hat{\bs{\theta}}_1$ and covariance matrix $(\mathbf{W}'\mathbf{R}^{-1}\mathbf{W} + \frac{\sigma_{e}^{2}}{\sigma_u^2}\mathbf{D})^{-1}\sigma^2_e$

## Breeding value model (pedigree based)

Suppose that $\mbf{u}$ in (\ref{eq.mixedModel}) is a vector of breeding values. Then, given a pedigree $\mbf{P}$ for the individuals, the covariance matrix of $\mbf{u}$ is

$$
\Var(\mbf{u}|\mbf{P}) = \mbf{A}\sigma^2_a,
$$

where $\mbf{A}$ is the additive genetic relationship matrix and $\sigma^2_a$ is the additive genetic variance.

## Genomic Models

Suppose SNP genotypes are available on each of the $n$ individuals at $k$ loci. Let $\mbf{M}$ denote the $n\times k$ matrix of SNP genotype covariates and $\bs{\alpha}$ denote their effects. Although from a frequentist point of view $\bs{\alpha}$ is a fixed effect, for Bayesian analyses, $\bs{\alpha}$ is often assigned a normal prior with null mean and covariance matrix: $\Var(\bs{\alpha}) = \mbf{I}\sigma^2_{\alpha}$. Then, the breeding values can be modeled as: 

\begin{equation*}
\mbf{u} = \mbf{M}\bs{\alpha}.
\end{equation*}

Assuming that the $\mbf{Z}$ matrix in (\ref{eq.mixedModel}) is an identity matrix, the model for the phenotypes can be wrtten as either:

\begin{equation}
\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \mathbf{M}\bs{\alpha} + \mathbf{e},
\label{eq.MEM}
\end{equation}

or as 

\begin{equation}
\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \mbf{u} + \mathbf{e}.
\label{eq.BVM}
\end{equation}

In (\ref{eq.BVM}), conditional on the SNP genotypes, the covariance matrix of $\mbf{u}$ is: 

$$
\begin{align}
\Var(\mbf{u}|\mbf{M}) &= \mbf{MM}'\sigma^2_{\alpha}\\
                      &= \frac{\mbf{MM}'}{k\overline{2pq}}k\overline{2pq}\sigma^2_{\alpha}\\
                      &= \mbf{G}\sigma^2_u,
\end{align}
$$

where $\mbf{G} = \frac{\mbf{MM}'}{k\overline{2pq}}$ and $\sigma^2_u = k\overline{2pq}\sigma^2_{\alpha}$. The model given by (\ref{eq.BVM}) is a genomic breeding value model. 

Assuming, for simplicity that $\mbf{R} = \mbf{I}$, mixed model equations (MME) that correspend to model (\ref{eq.MEM}) is:

\begin{equation}
\begin{bmatrix}
\mathbf{X}'\mathbf{X} & \mathbf{X}'\mathbf{M} \\
\mathbf{M}'\mathbf{X} & \mathbf{M}'\mathbf{M} + \frac{\sigma_{e}^{2}}{\sigma_{\alpha}^2}\mathbf{I}
\end{bmatrix}
\begin{bmatrix}
\hat{\bs{\beta}}\\
\hat{\bs{\alpha}}
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{X}'\mathbf{y}\\
\mathbf{M}'\mathbf{y}
\end{bmatrix},
\label{eq.MMEMEM}
\end{equation}

and the MME corresponding to (\ref{eq.BVM}) is:

\begin{equation}
\begin{bmatrix}
\mathbf{X}'\mathbf{X} & \mathbf{X}' \\
\mathbf{X} & \mathbf{I} + \frac{\sigma_{e}^{2}}{\sigma_u^2}\mathbf{G}^{-1}
\end{bmatrix}
\begin{bmatrix}
\hat{\bs{\beta}}\\
\hat{\mbf{u}}
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{X}'\mathbf{y}\\
\mathbf{y}
\end{bmatrix},
\label{MMEBVM}
\end{equation}

where $\mbf{G} = \frac{\mbf{MM}'}{k\overline{2pq}}$ and $\sigma^2_u = k\overline{2pq}\sigma^2_{\alpha}$.